### 1.1 Library Import

In [10]:
import pandas as pd
import numpy as np
import os
import pickle as pk
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression


### 1.2 Pickling


In [8]:
def dump_var(var,var_name):
    dump_path=r'.'
    dump_var=dump_path+"\\"+var_name+".pickle"
    var_file=open(dump_var,"wb")
    pk.dump(var,var_file)
    var_file.close()
    
def load_var(var_name):
    load_path=r'.'
    load_var=load_path+"\\"+var_name+".pickle"
    with open(load_var,"rb") as var_file:
        print(load_var)
        content=pk.load(var_file)
    var_file.close()
    return content

### 2.1 Data Load

In [ ]:
story = pd.read_csv(r'story.csv', header=None, error_bad_lines=False)
user_favorite_author = pd.read_csv(r'author_favorites.csv', header=None)

In [ ]:
story.head()

In [ ]:
columns=['id','fandom_id','user_id','rating_id','language_id',
         'ff_story_id','title','chapters','words',
         'reviews','favorites','followers','date_published',
         'date_updated','is_complete']
story.columns=columns
story.head()

### 2.2 Aggregated GOT Tables


In [ ]:
game_ot=story[story.fandom_id == 8477][['user_id','id','reviews','chapters','favorites','followers','words']].groupby('user_id').agg({
    'id':['count'],
    'reviews':['sum'],
    'chapters':['sum'],
    'favorites':['sum','mean'],
    'followers':['sum','mean'],
    'words':['sum']
})
game_ot.columns = ["_".join(x) for x in game_ot.columns.ravel()]
GOT=game_ot.reset_index()

In [ ]:
user_id_GOT=sorted(np.array(GOT['user_id']))
user_authors=np.array(user_favorite_author)

### 2.3 Extract list of users and favourited authors from user_author table (very time consuming)

In [ ]:
user_author_got=np.array( [[users[0],users[1]] for users in user_authors if users[0] in user_id_GOT and users[1] in user_id_GOT])

### 2.4 Unique number of users and authors favourited by users

In [ ]:
print("Users: "+str(len(np.unique(user_author_got[:,0]))))
print("Authors favorited by users: "+str(len(np.unique(user_author_got[:,1]))))

### 3.0 PageRank

In [ ]:
kv_list=np.array(user_author_got)

s1=set(kv_list[:,0])
s2=set(kv_list[:,1])
s=s1.union(s2)
AuthorList=list(s)
AuthorList.sort()
AuthorDict={}
for i in range(len(AuthorList)):
    AuthorDict[AuthorList[i]]=i

NormKVList=np.zeros(shape=(len(kv_list),2))
for i in range(len(kv_list)):
    NormKVList[i][0]=AuthorDict[kv_list[i][0]]
    NormKVList[i][1]=AuthorDict[kv_list[i][1]]

# Define a zero Matrix with length as number of unique journals
len_NormKV_list=len(NormKVList)
AdjMat=np.zeros(shape=(len(AuthorList),len(AuthorList)))

for i in range(len_NormKV_list):
    AdjMat[int(NormKVList[i][1]),int(NormKVList[i][0])]=1

# Get column sums
ColSum=AdjMat.sum(axis=0)
ColSum1=np.array(ColSum)

# Set the column sums with value 0 to -1 (to avoid 0/0 and X/0 divisions)
ColSum1[ColSum1==0]=-1

# Normaliza the matrix
AdjMat_norm=np.true_divide(AdjMat, ColSum1)

# Dangling Nodes
Dang=np.array(ColSum1)
Dang[Dang>0]=0
Dang[Dang==-1]=1

# Teleoport Vector
TeleportVector=np.full(len(AuthorList),(1/len(AuthorList)))
InitialStartVector=np.array(TeleportVector)

# PageRank Parameters
alpha=0.85
e=0.0001
H=np.array(AdjMat_norm)
pi_0=np.array(InitialStartVector)
d=np.array(Dang)
pi_k=np.array(pi_0)
pi_k1=np.empty(len(pi_0))
run_flag=True
a=np.array(TeleportVector)

countIterations=0
run_flag=True
def l1(pi_1,pi_2):
    return np.round(np.linalg.norm((pi_1-pi_2),1),5)

    ### l1 output rounded off to 5 digits as e=0.00001

while(run_flag):
    countIterations+=1
    pi_k1=np.array(alpha*(H.dot(pi_k))+(alpha*(d.dot(pi_k))+(1-alpha))*a) #Pagerank equation
    if l1(pi_k,pi_k1)<e:
        run_flag=False
    pi_k=np.array(pi_k1)


EF=100*np.true_divide(H.dot(pi_k),np.sum(H.dot(pi_k)))

# Generate Index
EF_index=np.arange(0,len(AuthorList))

# Concat two arrays
EF_final=np.vstack((EF, EF_index))

EF_finalT=EF_final.T

# Sort the journals and get top 20
sortedEF_finalT=EF_finalT[EF_finalT[:,0].argsort()[::-1]][0:20]

def get_key(val):
    for key, value in AuthorDict.items():
        if val == value:
            return key

# Get Authors from Dictionary
for i in range(len(sortedEF_finalT)):
    print("Score: "+str(sortedEF_finalT[i][0])+" Author:"+str(get_key(sortedEF_finalT[i][1])))

In [ ]:
pageranks=np.zeros(shape=(len(EF_finalT),2))

for i in range(len(EF_finalT)):
    pageranks[i][0]=get_key(EF_finalT[i][1])
    pageranks[i][1]=EF_finalT[i][0]

### 3.2 If PageRanks are computed earlier, then load the file directly

In [ ]:
pageranks=pd.read_csv('pagerank.csv')
pageranks=pageranks.drop(columns=['Unnamed: 0'])
pgs=pd.DataFrame(pageranks,columns=['user_id','score']).astype({'user_id':np.int64,'score':float})
#merged_scores=GOT.merge(pgs,how='left',on='user_id')

In [ ]:
merged_scores=GOT.merge(pagerank,how='left',on='user_id')
merged_scores[(merged_scores['scores']!=0) & (merged_scores['scores']!=np.nan)]

### 3.3 Plot Histogram

In [ ]:
pg_df=pd.read_csv(r'C:\Users\ajink\Documents\drg\data\pg.csv')
pg_df.columns=['id','rank']
pg_df1=pg_df[pg_df['rank']!=0]
fig, ax=plt.subplots(1,1,figsize=(10,8))
ax.set_ylabel("Frequency (log)")
ax.set_xlabel("Pageranks")
ax.set_yscale('log')
ax.set_xlim(0, 0.5)
ax.hist(pg_df1['rank'],bins=100,color='Green')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.show()

### 3.4 Filter the records based on nulls and zeros scores

In [ ]:
GOT_filtered=merged_scores[(merged_scores['score']!=0) & (merged_scores['score'].notnull())]

In [ ]:
GOT_filtered['story_chapter_product']=GOT_filtered['id_count']*GOT_filtered['chapters_sum']

In [ ]:
GOT_filtered.head()

### 4.1 Regression

In [ ]:
feature_list=['reviews_sum','favorites_sum','words_sum','story_chapter_product']

In [ ]:

y_tr=np.array(GOT_filtered_1['score'])
results=[]
slope=[]
intercept=[]
x_tr=[]
r2=[]

for index, x_data in enumerate(feature_list):
    x_tr.append(np.array(GOT_filtered_1[[x_data]]))

for index, x_data in enumerate(feature_list):
    reg = LinearRegression().fit(x_tr[index], y_tr)
    r2.append(reg.score(x_tr[index], y_tr))
    slope.append(reg.coef_)
    intercept.append(reg.intercept_)


fig, axs = plt.subplots(2,2, figsize=(20, 20))
for index, x_data in enumerate(feature_list):
    axs[int(index/2)][index%2].scatter(x_tr[index],y_tr,c='c',alpha=0.33)
    y = slope[index]*x_tr[index] + intercept[index]
    axs[int(index/2)][index%2].plot(x_tr[index], y, 'm-')
    axs[int(index/2)][index%2].set_title(" PageRank ~ "+str(x_data)+", R-Squared="+str(round(r2[index],5)))
    axs[int(index/2)][index%2].set_xlabel(x_data)
    axs[int(index/2)][index%2].set_ylabel('Scores')
plt.show()    


